### Food for thought: how to eat sustainably

#### Introduction

In the face of growing concerns about environmental sustainability, understanding the impact of our food choices has become paramount. This data story aims to scientifically examine the sustainability of various food sources, taking into account crucial factors like CO2 emissions, water usage, and the distinction between local and global production. By presenting this information concisely and incorporating charts and plots, our goal is to provide accessible insights that empower individuals to make informed decisions for more sustainable food consumption practices.

A significant debate centers around whether local or global food production is more environmentally preferable. Advocates of the local perspective argue that transportation emissions significantly contribute to the carbon footprint of our food. They stress the importance of reducing the distance between farm and table to enhance sustainability. Conversely, proponents of the global perspective emphasize evaluating the environmental impact of different food types, such as animal products versus plant-based alternatives. Metrics like calories or protein produced per unit of emissions are considered crucial in this evaluation.

To comprehensively assess the sustainability of food sources, this data story will analyze a range of common options. Environmental impact will be examined across multiple dimensions, including CO2 emissions, water usage, and the balance between local and non-local production.

#### Dataset and preprocessing

The dataset we used can be downloaded from https://www.science.org/doi/10.1126/science.aaq0216. It is called 'aaq0216_datas1.xls'. The dataset encompasses variables such as land use (in m2), greenhouse gas emissions (in kg CO2-eq), eutrophication (in kg PO43-eq) and freshwater (in liters), for 43 different agricultural products. The dataset is an xls file containing multiple Excel sheets. 

To look at the global totals of Greenhouse gas emissions for each product, for example, we can look at the following dataframe:

In [1]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

df_ghg = pd.read_excel('dataset1.xls', sheet_name=2, skiprows=2, nrows=43, usecols="A, F:L")
df_ghg['Total'] = df_ghg[['LUC', 'Feed', 'Farm', 'Processing', 'Transport', 'Packging', 'Retail']].sum(axis=1)
df_ghg.head()

,Product,LUC,Feed,Farm,Processing,Transport,Packging,Retail,Total
0,Wheat & Rye (Bread),0.100,0.0,0.847,0.217,0.129,0.090,0.058,1.441
1,Maize (Meal),0.315,0.0,0.475,0.052,0.060,0.060,0.026,0.988
2,Barley (Beer),0.009,0.0,0.176,0.128,0.035,0.497,0.264,1.109
3,Oatmeal,0.001,0.0,1.370,0.042,0.067,0.066,0.029,1.575
4,Rice,-0.022,0.0,3.553,0.065,0.096,0.084,0.063,3.839


Here, we can see that the greenhouse gas emissions are divided into different stages of the production and distribution of the product. We added another column to see the total amount of greenhouse gas emissions, as that is often the most important metric to compare.

#### Visualisations

##### Visualisation 1: bar charts of the amount of freshwater used for each product, measured in liters per NU (nutritional unit) or FU (fixture unit)

In [2]:
df_land = pd.read_excel('dataset1.xls', sheet_name=1, skiprows=2,nrows=43)
colormap = {
    'Wheat & Rye (Bread)':'darkblue',
    'Maize (Meal)':'darkblue',
    'Barley (Beer)':'darkblue',
    'Oatmeal':'darkblue',
    'Rice':'darkblue',
    'Potatoes':'darkblue',
    'Cassava':'darkblue',
    'Cane Sugar':'magenta',
    'Beet Sugar':'magenta',
    'Other Pulses':'magenta',
    'Peas':'brown',
    'Nuts':'brown',
    'Groundnuts':'brown',
    'Soymilk':'purple',
    'Tofu':'purple',
    'Soybean Oil':'black',
    'Palm Oil':'black',
    'Sunflower Oil':'black',
    'Rapeseed Oil':'black',
    'Olive Oil':'black',
    'Tomatoes':'green',
    'Onions & Leeks':'green',
    'Root Vegetables':'green',
    'Brassicas':'green',
    'Other Vegetables':'green',
    'Citrus Fruit':'orange',
    'Bananas':'orange',
    'Apples':'orange',
    'Berries & Grapes':'orange',
    'Wine':'orange',
    'Other Fruit':'orange',
    'Coffee':'gold',
    'Dark Chocolate':'gold',
    'Bovine Meat (beef herd)':'red',
    'Bovine Meat (dairy herd)':'red',
    'Lamb & Mutton':'red',
    'Pig Meat':'red',
    'Poultry Meat':'red',
    'Milk':'grey',
    'Cheese':'grey',
    'Eggs':'grey',
    'Fish (farmed)':'blue',
    'Crustaceans (farmed)':'blue'
}
fig = px.histogram(df_land, x='Product', y='Mean.5', color='Product', color_discrete_map=colormap)
fig.update_layout(
    font=dict(
        family='Lato, "Helvetica Neue", Helvetica, Arial, "Liberation Sans", sans-serif',
        size=13,
    ),
    yaxis=dict(title='Average Freshwater Withdrawals (L/NU)'),
    xaxis=dict(title='Products'),
    title='Freshwater Withdrawals (L/NU)',
    showlegend=False,
)
values = ['starchy', 'sugars', 'legume', 'vegan alt', 'oils', 'vegetables', 'fruits','proc nut', 'meat', 'animal prod', 'fish']
colors = ['darkblue', 'magenta', 'green', 'purple', 'black', 'green', 'orange','gold', 'red', 'grey', 'blue']


for i in range(len(values)-1):
    fig.add_annotation(
        x=1.05, y=1 -i*0.11,
        xref='paper', yref='paper',
        text=values[i],
        showarrow=False,
        font=dict(color='white'),
        borderwidth=1,
        bgcolor= colors[i]
    )
fig.show()

fig1: Amount of freshwater withdrawals per nutritional units

In figure 1 you can see the different amount of liters water used for a nutritional unit. In the figure you can also see that nuts and cheese use alot of water for the amount of nutritions that they give. Most of the meat substitutes are made from wheat, soy and different kinds of fungi(https://www.milieucentraal.nl/eten-en-drinken/milieubewust-eten/vleesvervangers/). The figure shows that there is a substatial difference between amount of water used for the different kinds of meat and the water usage from wheat and soymilk. The only meat that doesn't use more than double the amount of water as the wheat is poultry meat. 

In [3]:
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd

df_land = pd.read_excel('dataset1.xls', sheet_name=0, skiprows=2,nrows=43)

fig = px.histogram(df_land, x='Product', y='Mean.5', color='Product', color_discrete_map=colormap)
fig.update_layout(
    font=dict(
        family='Lato, "Helvetica Neue", Helvetica, Arial, "Liberation Sans", sans-serif',
        size=13,
    ),
    xaxis=dict(title='Average Freshwater Withdrawals (L/FU)'),
    yaxis=dict(title='Products'),
    title = 'Freshwater Withdrawals (L/FU)',
    showlegend=False
)
values = ['starchy', 'sugars', 'legume', 'vegan alt', 'oils', 'vegetables', 'fruits','proc nut', 'meat', 'animal prod', 'fish']
colors = ['darkblue', 'magenta', 'green', 'purple', 'black', 'green', 'orange','gold', 'red', 'grey', 'blue']


for i in range(len(values)-1):
    fig.add_annotation(
        x=1.05, y=1 -i*0.11,
        xref='paper', yref='paper',
        text=values[i],
        showarrow=False,
        font=dict(color='white'),
        borderwidth=1,
        bgcolor= colors[i]
    )
fig.show()

fig2: Amount of freshwater withdrawals per Fixture unit

In [4]:
#Mauro
non_vegan_products = ['Barley (Beer)','Cane Sugar','Milk','Cheese','Eggs','Fish (farmed)','Crustaceans (farmed)','Bovine Meat (beef herd)','Bovine Meat (dairy herd)','Lamb & Mutton','Pig Meat','Poultry Meat']
meat_products = ['Bovine Meat (beef herd)', 'Bovine Meat (dairy herd)', 'Lamb & Mutton', 'Pig Meat','Poultry Meat','Fish (farmed)','Crustaceans (farmed)']
dairy_products = ['Milk','Cheese','Eggs']
other_non_vegan_products = ['Cane Sugar','Crustaceans (farmed)','Barley (Beer)']

df_all_products = pd.read_excel("dataset1.xls", sheet_name="Results - Retail Weight", skiprows=2,nrows=43, index_col=None, na_values=["NA"])

#non vegan products
non_vegan = df_all_products["Product"].isin(non_vegan_products)
meat = df_all_products["Product"].isin(meat_products)
dairy = df_all_products["Product"].isin(dairy_products)
other_non_vegan = df_all_products["Product"].isin(other_non_vegan_products)

df_non_vegan_products = df_all_products[non_vegan]
df_meat_products = df_all_products[meat]
df_dairy_products = df_all_products[dairy]
df_other_non_vegan_products = df_all_products[other_non_vegan]

#non vegan
df_non_vegan_products_land_use = df_non_vegan_products.iloc[:, [0, 3]]
df_non_vegan_products_ghg_2013 = df_non_vegan_products.iloc[:, [0, 9]]
df_non_vegan_products_ghg_2007 = df_non_vegan_products.iloc[:, [0, 15]]
df_non_vegan_products_acid = df_non_vegan_products.iloc[:, [0, 21]]
df_non_vegan_products_eutro = df_non_vegan_products.iloc[:, [0, 27]]
df_non_vegan_products_fresh_water_withdraw = df_non_vegan_products.iloc[:, [0, 33]]
df_non_vegan_products_stress_water_use = df_non_vegan_products.iloc[:, [0, 39]]

#meat products
df_meat_products_land_use = df_meat_products.iloc[:, [0, 3]]
df_meat_products_ghg_2013 = df_meat_products.iloc[:, [0, 9]]
df_meat_products_ghg_2007 = df_meat_products.iloc[:, [0, 15]]
df_meat_products_acid = df_meat_products.iloc[:, [0, 21]]
df_meat_products_eutro = df_meat_products.iloc[:, [0, 27]]
df_meat_products_fresh_water_withdraw = df_meat_products.iloc[:, [0, 33]]
df_meat_products_stress_water_use = df_meat_products.iloc[:, [0, 39]]
#dairy
df_dairy_products_land_use = df_dairy_products.iloc[:, [0, 3]]
df_dairy_products_ghg_2013 = df_dairy_products.iloc[:, [0, 9]]
df_dairy_products_ghg_2007 = df_dairy_products.iloc[:, [0, 15]]
df_dairy_products_acid = df_dairy_products.iloc[:, [0, 21]]
df_dairy_products_eutro = df_dairy_products.iloc[:, [0, 27]]
df_dairy_products_fresh_water_withdraw = df_dairy_products.iloc[:, [0, 33]]
df_dairy_products_stress_water_use = df_dairy_products.iloc[:, [0, 39]]

#other non vegan
df_other_non_vegan_products_land_use = df_other_non_vegan_products.iloc[:, [0, 3]]
df_other_non_vegan_products_ghg_2013 = df_other_non_vegan_products.iloc[:, [0, 9]]
df_other_non_vegan_products_ghg_2007 = df_other_non_vegan_products.iloc[:, [0, 15]]
df_other_non_vegan_products_acid = df_other_non_vegan_products.iloc[:, [0, 21]]
df_other_non_vegan_products_eutro = df_other_non_vegan_products.iloc[:, [0, 27]]
df_other_non_vegan_products_fresh_water_withdraw = df_other_non_vegan_products.iloc[:, [0, 33]]
df_other_non_vegan_products_stress_water_use = df_other_non_vegan_products.iloc[:, [0, 39]]


#vegan products
grains_products = ['Wheat & Rye (Bread)', 'Maize (Meal)', 'Oatmeal', 'Rice']
vegetables_products = ['Potatoes', 'Cassava', 'Tomatoes', 'Onions & Leeks', 'Root Vegetables', 'Brassicas', 'Other Vegetables']
fruits_products = ['Citrus Fruit', 'Bananas', 'Apples', 'Berries & Grapes', 'Other Fruit']
legumes_products = ['Other Pulses', 'Peas', 'Nuts', 'Groundnuts']
plant_based_alternatives_products = ['Soymilk', 'Tofu']
oils_products = ['Soybean Oil', 'Sunflower Oil', 'Olive Oil']
others_products = ['Coffee', 'Dark Chocolate']
vegan = df_all_products['Product'].isin(non_vegan_products)
grains = df_all_products["Product"].isin(grains_products)
vegetables = df_all_products["Product"].isin(vegetables_products)
fruits = df_all_products["Product"].isin(grain)
legumes = df_all_products["Product"].isin(legumes_products)
plant_based_alternatives = df_all_products["Product"].isin(plant_based_alternatives_products)


df_vegan_products = df_all_products[vegan]
df_grains_products = df_all_products[grains]
df_vegetables_products = df_all_products[vegetables]
df_fruits_products = df_all_products[fruits]
df_legumes_products = df_all_products[legumes]
df_plant_based_alternatives_products = df_all_products[plant_based_alternatives]
df_oils_products = df_all_products[oils]
df_others_products = df_all_products[others]

#vegan
df_vegan_products_land_use = df_vegan_products.iloc[:, [0, 3]]
df_vegan_products_ghg_2013 = df_vegan_products.iloc[:, [0, 9]]
df_vegan_products_ghg_2007 = df_vegan_products.iloc[:, [0, 15]]
df_vegan_products_acid = df_vegan_products.iloc[:, [0, 21]]
df_vegan_products_eutro = df_vegan_products.iloc[:, [0, 27]]
df_vegan_products_fresh_water_withdraw = df_vegan_products.iloc[:, [0, 33]]
df_vegan_products_stress_water_use = df_vegan_products.iloc[:, [0, 39]]





KeyError: "None of [Index(['Wheat & Rye (Bread)', 'Maize (Meal)', 'Oatmeal', 'Rice'], dtype='object')] are in the [columns]"

##### Visualisation 2: sunburst charts

In [6]:
#sunbursts

#sunbursts

def create_sunburst_figure(title, values):
    colors = ['darkgreen', 'darkred'] * 2  
    fig = go.Figure(go.Sunburst(
        labels=labels,
        parents=parents,
        values=values,
        marker=dict(colors=colors),
    ))
    
    fig.update_layout(title=title)
    fig.show()
categories = ['Land use', 'GHG emission 2013', 'GHG emission 2007', 'Acidification', 'Eutrophication', 'Fresh Water Withdrawal', 'Stress Water Use']

labels = ["Total", "Non-vegan","Vegan","Meat", "Dairy products","Other non-vegan Products","Vegetables", "Fruits", "Legumes", "Plant-based alternatives", "Oils", "Other vegan products"] + list(df_meat_products["Product"]) + list(df_dairy_products["Product"]) + list(df_other_non_vegan_products["Product"]) + list(df_vegetables_products["Product"]) + list(df_fruits_products["Product"]) + list(df_legumes_products["Product"]) + list(df_plant_based_alternatives_products["Product"]) + list(df_oils_products["Product"]) + list(df_others_products["Product"])
parents = ["", "Total","Total"] + ["Non-vegan"] * 3 + ["Vegan"] * 6 + ["Meat"] * len(df_meat_products) + ["Dairy products"] * len(df_dairy_products) + ["Other non-vegan Products"] * len(df_other_non_vegan_products)+ ["Vegetables"] * len(df_vegetables_products) + ["Fruits"] * len(df_fruits_products) + ["Legumes"] * len(df_legumes_products) + ["Plant-based alternatives"] * len(df_plant_based_alternatives_products) + ["Oils"] * len(df_oils_products) + ["Other vegan products"] * len(df_others_products)

# Land use
values = [0] * 4 + list(df_non_vegan_products_land_use["Mean"]) + list(df_vegan_products_land_use["Mean"])

create_sunburst_figure("Land use", values)

# GHG emission 2013
values = [0] * 4 + list(df_non_vegan_products_ghg_2013["Mean.1"]) + list(df_vegan_products_ghg_2013["Mean.1"])
create_sunburst_figure("GHG emission 2013", values)

# GHG emission 2007
values = [0] * 3 + list(df_non_vegan_products_ghg_2007["Mean.2"]) + list(df_vegan_products_ghg_2007["Mean.2"])
create_sunburst_figure("GHG emission 2007", values)

# Acidification
values = [0] * 3 + list(df_non_vegan_products_acid["Mean.3"]) + list(df_vegan_products_acid["Mean.3"])
create_sunburst_figure("Acidification", values)

# Eutrophication
values = [0] * 3 + list(df_non_vegan_products_eutro["Mean.4"]) + list(df_vegan_products_eutro["Mean.4"])
create_sunburst_figure("Eutrophication", values)

# Fresh Water Withdrawal
values = [0] * 3 + list(df_non_vegan_products_fresh_water_withdraw["Mean.5"]) + list(df_vegan_products_fresh_water_withdraw["Mean.5"])
create_sunburst_figure("Fresh Water Withdrawal", values)

# Stress Water Use
values = [0] * 3 + list(df_non_vegan_products_stress_water_use["Mean.6"]) + list(df_vegan_products_stress_water_use["Mean.6"])
create_sunburst_figure("Stress Water Use", values)


NameError: name 'df_vegetables_products' is not defined

##### Visualisation 3: bar plots

In [14]:
#Bar plots
categories = ['Land use', 'GHG emission 2013', 'GHG emission 2007', 'Acidification', 'Eutrophication', 'Fresh Water Withdrawal', 'Stress Water Use']
vegan_values = [df_vegan_products_land_use["Mean"].sum(),
                df_vegan_products_ghg_2013["Mean.1"].sum(),
                df_vegan_products_ghg_2007["Mean.2"].sum(),
                df_vegan_products_acid["Mean.3"].sum(),
                df_vegan_products_eutro["Mean.4"].sum(),
                df_vegan_products_fresh_water_withdraw["Mean.5"].sum(),
                df_vegan_products_stress_water_use["Mean.6"].sum()]
non_vegan_values = [df_non_vegan_products_land_use["Mean"].sum(),
                    df_non_vegan_products_ghg_2013["Mean.1"].sum(),
                    df_non_vegan_products_ghg_2007["Mean.2"].sum(),
                    df_non_vegan_products_acid["Mean.3"].sum(),
                    df_non_vegan_products_eutro["Mean.4"].sum(),
                    df_non_vegan_products_fresh_water_withdraw["Mean.5"].sum(),
                    df_non_vegan_products_stress_water_use["Mean.6"].sum()]
values = ['Land Use (m2/FU)', 'GHG Emissions (kg CO2eq/FU)', 'GHG Emissions (kg CO2eq/FU)', 'Acidifying Emissions (g SO2eq/FU)', 'Eutrophying Emissions (g PO43-eq/FU)', 'Freshwater Withdrawals (L/FU)', 'Stress-Weighted Water Use (L/FU)']


for category, vegan_value, non_vegan_value in zip(categories, vegan_values, non_vegan_values):
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=['Vegan', 'Non-vegan'],
        y=[vegan_value, non_vegan_value],
        name=category,
        marker_color=['darkgreen','darkred']  
    ))

    fig.update_layout(
        title=f'Environmental Impact: {category}',
        xaxis_title='Product Type',
        yaxis_title= values[categories.index(category)]
    )
    fig.show()

NameError: name 'df_vegan_products_land_use' is not defined

##### Visualisation 4: interactive bar chart with dropdown menu

In [15]:
df_land = pd.read_excel('dataset1.xls', sheet_name=2, skiprows=2, nrows=43, usecols="A, C:E")
df_land['Total'] = df_land[['Arable', 'Fallow', 'Perm Past']].sum(axis=1)

df_ghg = pd.read_excel('dataset1.xls', sheet_name=2, skiprows=2, nrows=43, usecols="A, F:L")
df_ghg['Total'] = df_ghg[['LUC', 'Feed', 'Farm', 'Processing', 'Transport', 'Packging', 'Retail']].sum(axis=1)

df_eutr = pd.read_excel('dataset1.xls', sheet_name=2, skiprows=2, nrows=43, usecols="A, N")
df_eutr.rename(columns={"Total.1": "Total"}, inplace=True)

df_fresh = pd.read_excel('dataset1.xls', sheet_name=2, skiprows=2, nrows=43, usecols="A, O")
df_fresh.rename(columns={"Total.2": "Total"}, inplace=True)

df_list = [df_land, df_ghg, df_eutr, df_fresh]

fig = px.bar(df_land, x="Product", y="Total")

updatemenus = [
    {
        "buttons": [
            {
                "label": df_name,
                "method": "update",
                "args": [
                    {"y": [df['Total']]},
                    {"yaxis": {"title": {"text": df_name}}}
                ],
            }
            for df, df_name in zip(df_list, ['Land use', 'Greenhouse gas emissions', 'Eutrophication', 'Freshwater use'])
        ],
        "x": 0,
        "y": 1.2,
        "buttons": [
            {
                "label": f"{df_name}: {column}",
                "method": "update",
                "args": [
                    {"y": [df[column]]},
                    {"yaxis": {"title": {"text": f"{df_name}: {column}"}}}
                ],
            }
            for df, df_name in zip(df_list, ['Land use', 'Greenhouse gas emissions', 'Eutrophication', 'Freshwater use'])
            for column in df.columns[1:]
        ],
        "x": 0,
        "y": 1.2,
    }
]

fig.update_layout(updatemenus=updatemenus, 
        font=dict(family='Lato, "Helvetica Neue", Helvetica, Arial, "Liberation Sans", sans-serif', size=10),
        width=1000, height=500)
fig.show()